# YouTube Trending Videos Statistical Analysis

## INTRODUCTION

#### to be edited</b>
<b>https://www.kaggle.com/rsrishav/youtube-trending-video-dataset

In [1]:
# Importing libraries
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import scipy
from scipy import stats
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import ttest_1samp

%matplotlib inline

In [2]:
'''
Later on we can also remove the ones we wouldn't use
'''
# Importing dataset
data_gb = pd.read_csv("../data/GB_youtube_trending_data.csv",sep=",")  #uk  
data_fr = pd.read_csv("../data/FR_youtube_trending_data.csv",sep=",")  #france
data_us = pd.read_csv("../data/US_youtube_trending_data.csv",sep=",")  #usa 
data_ru = pd.read_csv("../data/RU_youtube_trending_data.csv",sep=",")  #russia
data_de = pd.read_csv("../data/DE_youtube_trending_data.csv",sep=",")  #germany 
data_ca = pd.read_csv("../data/CA_youtube_trending_data.csv",sep=",")  #canada   
data_kr = pd.read_csv("../data/KR_youtube_trending_data.csv",sep=",")  #southkorea   
data_jp = pd.read_csv("../data/JP_youtube_trending_data.csv",sep=",")  #japan
data_br = pd.read_csv("../data/BR_youtube_trending_data.csv",sep=",")  #brazil
data_mx = pd.read_csv("../data/MX_youtube_trending_data.csv",sep=",")  #mexico

In [3]:
'''
This can stay for now to easily see what are our properties in the dataset. We can delete it once
we are familiar with the dataset
'''
brief_list_col = []
brief_list = []

for i in range(0,len(data_gb.columns)):
    brief_list_col.append(data_gb.columns[i])
    brief_list.append(data_gb[data_gb.columns[i]][0])

brief_df = pd.DataFrame(brief_list_col,columns=["Property"])
brief_df.insert(1,"Output",brief_list,True)
brief_df

,Property,Output
0,video_id,J78aPJ3VyNs
1,title,I left youtube for a month and THIS is what ha...
2,publishedAt,2020-08-11T16:34:06Z
3,channelId,UCYzPXprvl5Y-Sf0g4vX-m6g
4,channelTitle,jacksepticeye
5,categoryId,24
6,trending_date,2020-08-12T00:00:00Z
7,tags,jacksepticeye|funny|funny meme|memes|jacksepti...
8,view_count,2038853
9,likes,353790


In [4]:
'''
Adding json files
'''
gb_json = pd.read_json('../data/GB_category_id.json')
fr_json = pd.read_json('../data/FR_category_id.json')
us_json = pd.read_json('../data/US_category_id.json')
ru_json = pd.read_json('../data/RU_category_id.json')
de_json = pd.read_json('../data/DE_category_id.json')
ca_json = pd.read_json('../data/CA_category_id.json')
kr_json = pd.read_json('../data/KR_category_id.json')
jp_json = pd.read_json('../data/JP_category_id.json')
br_json = pd.read_json('../data/BR_category_id.json')
mx_json = pd.read_json('../data/MX_category_id.json')

In [5]:
'''
Defining function to flat json files to get rid of dictionary format for every country
'''

col_list = ["items","items_snippet"]

def flat_func(df,column):
    for column in col_list:
        flatten = pd.DataFrame(dict(df[column])).transpose()
        columns = [str(i) for i in flatten.columns]
        flatten.columns = [column + "_" + str(colname) for colname in columns]
        df = pd.concat([df,flatten],axis=1)
        df = df.drop(column,axis=1)
    return df

In [6]:
'''
Applying function
Creating flat databases
'''

gb_flat = flat_func(gb_json,col_list)[["items_id","items_snippet_title"]]
fr_flat = flat_func(fr_json,col_list)[["items_id","items_snippet_title"]]
us_flat = flat_func(us_json,col_list)[["items_id","items_snippet_title"]]
ru_flat = flat_func(ru_json,col_list)[["items_id","items_snippet_title"]]
de_flat = flat_func(de_json,col_list)[["items_id","items_snippet_title"]]
ca_flat = flat_func(ca_json,col_list)[["items_id","items_snippet_title"]]
kr_flat = flat_func(kr_json,col_list)[["items_id","items_snippet_title"]]
jp_flat = flat_func(jp_json,col_list)[["items_id","items_snippet_title"]]
br_flat = flat_func(br_json,col_list)[["items_id","items_snippet_title"]]
mx_flat = flat_func(mx_json,col_list)[["items_id","items_snippet_title"]]

In [7]:
'''
Dropping categoryId 29 for each country
'''

data_countries = [data_gb,data_fr,data_us,data_ru,data_de,data_ca,data_kr,data_jp,data_br,data_mx]

def category_remover(country):
    drop_idx = country[country["categoryId"]==29].index
    return drop_idx

data_gb = data_gb.drop(category_remover(data_gb))
data_fr = data_fr.drop(category_remover(data_fr))
data_us = data_us.drop(category_remover(data_us))
data_ru = data_ru.drop(category_remover(data_ru))
data_de = data_de.drop(category_remover(data_de))
data_ca = data_ca.drop(category_remover(data_ca))
data_kr = data_kr.drop(category_remover(data_kr))
data_jp = data_jp.drop(category_remover(data_jp))
data_br = data_br.drop(category_remover(data_br))
data_mx = data_mx.drop(category_remover(data_mx))

In [8]:
'''
Creating function to add trend name next to categoryId for every country
'''
def trend_adder(country_df,country_flat):
    trend = []
    for trend_id in country_df["categoryId"]:
        for i in range(0,len(country_flat)):
            if trend_id == int(country_flat["items_id"][i]):
                trend.append(country_flat["items_snippet_title"][i])

    country_df.insert(6,"trend",trend)     

In [9]:
'''
Applying Function
Adding trend column next to categoryId for every country
'''
trend_order_data_country = [data_gb,data_fr,data_us,data_ru,data_de,
                            data_ca,data_kr,data_jp,data_br,data_mx]

trend_order_data_flat = [gb_flat,fr_flat,us_flat,ru_flat,de_flat,
                         ca_flat,kr_flat,jp_flat,br_flat,mx_flat]

for i in range(0,len(trend_order_data_country)):
    trend_adder(trend_order_data_country[i],trend_order_data_flat[i])

In [10]:
'''
Scrapping the most updated row for each link.

Not the best way but kinda stuck while trying to retrieve local variables from function so this will
give us what we want for now..
'''
def get_recent_idx(country):
    latest_idx = []
    for link in np.array(country["video_id"].unique()):
        latest_row = country[(country["video_id"] ==
                                      link)][["view_count"]].nlargest(1,"view_count").index[0]
    
        latest_idx.append(latest_row)
    return country[country.index.isin(latest_idx)]
   
data_gb_unique = get_recent_idx(data_gb)
data_fr_unique = get_recent_idx(data_fr)
data_us_unique = get_recent_idx(data_us)
data_ru_unique = get_recent_idx(data_ru)
data_de_unique = get_recent_idx(data_de)
data_ca_unique = get_recent_idx(data_ca)
data_kr_unique = get_recent_idx(data_kr)
data_jp_unique = get_recent_idx(data_jp)
data_br_unique = get_recent_idx(data_br)
data_mx_unique = get_recent_idx(data_mx)

In [11]:
data_gb_unique.columns

Index(['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle',
       'categoryId', 'trend', 'trending_date', 'tags', 'view_count', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'description'],
      dtype='object')

## Statistical Analysis

#### Hypothesis 

View counts are largely similar for the US and UK

H0: mu UK == mu US

H1: mu UK != mu US

In [16]:
data_gb_unique['view_count'].mean()

2128238.6995253745

In [17]:
data_gb_unique['view_count'].std()

7214438.945745977

In [18]:
data_us_unique['view_count'].mean()

3041465.1761052245

In [19]:
data_us_unique['view_count'].std()

9549893.37796264

In [20]:
from scipy.stats import ttest_ind

ttest_ind(data_us_unique['view_count'], data_gb_unique['view_count'], equal_var=False)

Ttest_indResult(statistic=3.9923474414589903, pvalue=6.634332324974839e-05)

P value is much smaller than critical alpha value (0.05) therefore we can reject the null hypothesis and assume that the average number of views for a video in the US is statistically different from in the UK. 

#### Hypothesis 

The average number of likes on a trending video is greater than 200k 

H0: mean == 200k
H1: mean >200k 

In [14]:


us_likes = data_us_unique['likes']

In [15]:
alpha = 0.05

In [16]:
us_likes.mean()

156722.9039935458

In [17]:
us_likes.std()

552001.2496020874

In [19]:
mu = 200000
x_bar = us_likes.mean()
sigma = us_likes.std()
marg_error = sigma / len(us_likes)

In [20]:
z = (x_bar - mu) / marg_error
print(z)

-194.35448937359482


In [39]:
from scipy.stats import norm


In [37]:
p = norm.cdf(z)
print(1 - p)

1.0


In [33]:
#alternate test
p_values = scipy.stats.norm.sf(abs(z))
p_values

0.0

Such a small p value meens null hypothesis can be rejected and therefore we can assume mean is greater than 200k for likes. 

#### Confidence Intervals

In [36]:
# your code here
us_likes
us_likes_stdev = us_likes.std()
degree_freedom = len(us_likes) - 1
t = stats.t.interval(0.95, degree_freedom)[1]

# Terms
mean = us_likes.mean()
std_sample = np.sqrt(abs(np.sum((us_likes - mean)**2) / (1-len(us_likes))))
marg_of_error = std_sample / np.sqrt(len(us_likes))
con_int = [mean - (t * marg_of_error) , mean + (t * marg_of_error)]

print('With 95% confidence, the true mean lies in the inverval:', con_int)

With 95% confidence, the true mean lies in the inverval: [134982.77747488924, 178463.03051220236]


Possible hypothesis;

more popular the video more like/less dislike,
tbc